In [3]:
# API credentials and details
api_key = "7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x"
secret_key = "Oj3qyVUEufRp0cYtRMmWg9h5c3A"

In [7]:
import hmac
import hashlib
import json
import requests
import time

# ---- Config ----
base_url = "https://api.bitwyre.com"
# api_key = "your_api_key"
# secret_key = "your_secret_key"

# ---- Helpers ----

def double_dump(payload: dict | str | None) -> str:
    """
    Returns the double-dumped JSON string for Bitwyre checksum.
    """
    if payload is None or payload == {}:
        return json.dumps(json.dumps(""))  # double-dump of empty string
    elif isinstance(payload, dict):
        return json.dumps(json.dumps(payload, sort_keys=True))
    else:
        return json.dumps(json.dumps(payload))

def generate_signature(secret_key: str, uri_path: str, payload: dict | None) -> tuple:
    """
    Generates nonce, checksum, and signature for Bitwyre API.
    """
    nonce = time.time_ns()
    dd_payload = double_dump(json.dumps(payload) if isinstance(payload, dict) else payload)
    checksum = hashlib.sha256(dd_payload.encode()).hexdigest()
    nonce_checksum = hashlib.sha256(f"{nonce}{checksum}".encode()).hexdigest()
    signature = hmac.new(
        secret_key.encode(),
        (uri_path + nonce_checksum).encode(),
        hashlib.sha512
    ).hexdigest()
    return nonce, checksum, signature

def make_authenticated_request(method: str, uri_path: str, payload: dict | None, 
                              is_empty_payload_str: bool = False, 
                              is_wrapped_payload: bool = False,
                              is_cancel_request: bool = False):
    print(f"\n🔗 Calling {method.upper()} {uri_path}")

    # Generate signatures using existing logic
    signing_payload = "" if is_empty_payload_str else payload
    nonce, checksum, signature = generate_signature(secret_key, uri_path, signing_payload)

    headers = {
        "API-Key": api_key,
        "API-Sign": signature
    }

    # Add debug prints here
    print(f'nonce: {nonce}')
    print(f'checksum: {checksum}')
    print(f'signature: {signature}')
    print(f'headers: {headers}')
    print(f'params: {payload}')  # This shows the raw payload before processing

    # Special handling for cancel requests
    if is_cancel_request:
        # Calculate nonce checksum for verification
        nonce_check = hashlib.sha256(f"{nonce}{checksum}".encode()).hexdigest()
        print(f'nonce_check: {nonce_check}')
        
        # Build query parameters
        query_params = {
            "nonce": nonce,
            "checksum": checksum,
            "payload": payload["payload"],
            "instrument": payload["instrument"]
        }
        
        response = requests.request(
            method.upper(),
            f"{base_url}{uri_path}",
            headers=headers,
            params=query_params
        )
        
        # Print final constructed URL
        print("🌐 Final URL:", response.request.url)
        return response.status_code, response.json()

    # Existing handling for other request types
    if method.upper() == "GET":
        url = f"{base_url}{uri_path}?payload=&nonce={nonce}&checksum={checksum}"
        print("🔍 URL:", url)
        print("🧾 Headers:", headers)
        response = requests.get(url, headers=headers)

    else:
        url = f"{base_url}{uri_path}?nonce={nonce}&checksum={checksum}&{payload}"

        if isinstance(payload, dict) and "payload" in payload and isinstance(payload["payload"], str):
            # This is the multipart/form-data path
            print("📦 Multipart Payload (raw string):", payload["payload"])
            response = requests.request(
                method.upper(),
                url,
                headers=headers,
                files={"payload": (None, payload["payload"])}
            )
        else:
            # Standard JSON request
            final_payload = payload if is_wrapped_payload else (payload.get("payload", {}) if payload else {})
            print("📦 JSON Payload:", json.dumps(final_payload))
            response = requests.request(method.upper(), url, headers=headers, json=final_payload)

    print("🌐 Final URL:", url)  # 👈 Add this

    try:
        return response.status_code, response.json()
    except ValueError:
        return response.status_code, response.text
    
# ---- Endpoint Wrappers ----

def get_open_orders():
    uri_path = "/private/orders/open"
    return make_authenticated_request("GET", uri_path, {}, is_empty_payload_str=True)

def get_closed_orders():
    uri_path = "/private/orders/closed"
    return make_authenticated_request("GET", uri_path, None)

def create_order(side: int, instrument: str, price: str, ordtype: int, orderqty: str):
    uri_path = "/private/rust/order"
    payload = {
        "payload": {
            "side": side,
            "instrument": instrument,
            "price": price,
            "ordtype": ordtype,
            "orderqty": orderqty
        }
    }
    return make_authenticated_request("POST", uri_path, payload, is_wrapped_payload=True)

def cancel_order(order_ids: list, qtys: list, instrument: str):
    uri_path = "/private/orders/cancel"
    
    # 1. Create payload dictionary (not JSON string)
    payload_dict = {
        "order_ids": order_ids,
        "qtys": qtys,
        "instrument": instrument
    }
    
    # 2. Generate signatures using helper functions
    nonce, checksum, signature = generate_signature(
        secret_key=secret_key,
        uri_path=uri_path,
        payload=payload_dict  # Pass as dictionary
    )
    
    # 3. Create final parameters with JSON string payload
    params = {
        "nonce": nonce,
        "checksum": checksum,
        "payload": json.dumps(payload_dict),  # Convert to JSON string here
        "instrument": instrument
    }
    
    # 4. Send request directly
    headers = {"API-Key": api_key, "API-Sign": signature}
    response = requests.delete(
        f"{base_url}{uri_path}",
        headers=headers,
        params=params
    )
    return response.status_code, response.json()

In [8]:
status, result = get_open_orders()
print(f"✅ Status {status}\n📥 Response:", result)


🔗 Calling GET /private/orders/open
nonce: 1744357983410716124
checksum: 8bacbf15a33959ee066af7a77e9ef87e393725f4d28307d3f1496fb2c47b0d36
signature: 3e8426a622515f5265adc6770b95a73f190f9168ccd88c8d7d265bb3b0701493fc0357e8337b1ada5ec3e6ae2f0bca835ee1f9972cf0277a2c2fc92d70409c5a
headers: {'API-Key': '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x', 'API-Sign': '3e8426a622515f5265adc6770b95a73f190f9168ccd88c8d7d265bb3b0701493fc0357e8337b1ada5ec3e6ae2f0bca835ee1f9972cf0277a2c2fc92d70409c5a'}
params: {}
🔍 URL: https://api.bitwyre.com/private/orders/open?payload=&nonce=1744357983410716124&checksum=8bacbf15a33959ee066af7a77e9ef87e393725f4d28307d3f1496fb2c47b0d36
🧾 Headers: {'API-Key': '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x', 'API-Sign': '3e8426a622515f5265adc6770b95a73f190f9168ccd88c8d7d265bb3b0701493fc0357e8337b1ada5ec3e6ae2f0bca835ee1f9972cf0277a2c2fc92d70409c5a'}
🌐 Final URL: https://api.bitwyre.com/private/orders/open?payload=&nonce=1744357983410716124&checksum=8bacbf15a33959ee066af7a77e9ef8

In [5]:
status, result = get_closed_orders()
print(f"✅ Status {status}\n📥 Response:", result)


🔗 Calling GET /private/orders/closed
nonce: 1744357956004073468
checksum: 8bacbf15a33959ee066af7a77e9ef87e393725f4d28307d3f1496fb2c47b0d36
signature: f4ee4effddf455044a7aedff13865f602c3b2639adc8faf627b6defa8eea7ce44a0152923b476c66609ea172fe7029771ba4866000b7d9278338a9aab131d7a3
headers: {'API-Key': '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x', 'API-Sign': 'f4ee4effddf455044a7aedff13865f602c3b2639adc8faf627b6defa8eea7ce44a0152923b476c66609ea172fe7029771ba4866000b7d9278338a9aab131d7a3'}
raw payload: None
🌐 Final URL: https://api.bitwyre.com/private/orders/closed?nonce=1744357956004073468&checksum=8bacbf15a33959ee066af7a77e9ef87e393725f4d28307d3f1496fb2c47b0d36&payload=
📦 Response: {
  "error": [], 
  "meta": {
    "has_next": false, 
    "has_prev": false, 
    "next_page": null, 
    "page": 1, 
    "pages": 1, 
    "prev_page": null, 
    "total_count": 49
  }, 
  "result": {
    "btc_idr_spot": [
      {
        "AvgPx": "0", 
        "LastLiquidityInd": "0", 
        "LastPx": "1500",

In [17]:
status, response = create_order(
    side=1,
    instrument="doge_idr_spot",
    price="1500",
    ordtype=2,
    orderqty="2"
)
print(f"✅ Status {status}\n📥 Response:", response)


🔗 Calling POST /private/rust/order
nonce: 1744345904195369824
checksum: 921e47f9edc6da90bc0fe940c23b0293c68423ccfc774f07551f16e5607d7d7e
signature: aa7e0f799b9182ea829537f89c77cdbf51d433c53ab2bcbc10a402fab80646a81c940fc9706c7dfabcf86bf74fdcfa51a08f5b37a65a7a58e86faecae5ac30df
headers: {'API-Key': '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x', 'API-Sign': 'aa7e0f799b9182ea829537f89c77cdbf51d433c53ab2bcbc10a402fab80646a81c940fc9706c7dfabcf86bf74fdcfa51a08f5b37a65a7a58e86faecae5ac30df'}
params: {'payload': {'side': 1, 'instrument': 'doge_idr_spot', 'price': '1500', 'ordtype': 2, 'orderqty': '2'}}
📦 JSON Payload: {"payload": {"side": 1, "instrument": "doge_idr_spot", "price": "1500", "ordtype": 2, "orderqty": "2"}}
🌐 Final URL: https://api.bitwyre.com/private/rust/order?nonce=1744345904195369824&checksum=921e47f9edc6da90bc0fe940c23b0293c68423ccfc774f07551f16e5607d7d7e&{'payload': {'side': 1, 'instrument': 'doge_idr_spot', 'price': '1500', 'ordtype': 2, 'orderqty': '2'}}
✅ Status 200
📥 Respon

In [9]:
status, response = cancel_order(
    order_ids=["7b568c8c-0718-4094-afa5-c2bad621bd73"],
    qtys=[-1],
    instrument="doge_idr_spot"
)
print(f"Status: {status}, Response: {response}")

Status: 200, Response: {'error': [], 'result': []}


# Working Cancel Order

In [19]:
import hmac
import json
import time
from hashlib import sha256, sha512
import requests

# api_key = '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x'
# secret_key = 'Oj3qyVUEufRp0cYtRMmWg9h5c3A'
URL_API_BITWYRE = "https://api.bitwyre.com"

OPEN_ORDERS = "/private/orders/cancel"


def sign(secret_key: str, uri_path: str, payload: str) -> (int, str, str):
    nonce = time.time_ns()
    payload = json.dumps(payload)
    payload = json.dumps(payload)
    checksum = sha256(str(payload).encode("utf-8")).hexdigest()
    nonce_checksum = sha256(str(nonce).encode("utf-8") + str(checksum).encode("utf-8")).hexdigest()
    signature = hmac.new(
        secret_key.encode("utf-8"), uri_path.encode("utf-8") + nonce_checksum.encode("utf-8"), sha512
    ).hexdigest()
    return (nonce, checksum, signature)


def compute_nonce_checksum(nonce, checksum):
    # nonce_checksum is SHA256 of nonce + checksum
    nonce_checksum = sha256(str(nonce).encode("utf-8") + str(checksum).encode("utf-8")).hexdigest()
    return nonce_checksum


def compute_signature(api_secret, uri_path, nonce_checksum):
    # signature is HMAC-SHA512 of uri_path + nonce_checksum with api_secret as key
    signature = hmac.new(
        api_secret.encode("utf-8"), uri_path.encode("utf-8") + nonce_checksum.encode("utf-8"), sha512
    ).hexdigest()
    return signature


payload = {'order_ids': ['5b88198c-3894-4748-8e35-31d9099499f6'], 'qtys': [1], 'instrument': 'doge_idr_spot'}
payload = json.dumps(payload)

uri_path = OPEN_ORDERS

(nonce, checksum, signature) = sign(secret_key, uri_path, payload)
nonce_check = compute_nonce_checksum(nonce, checksum)
signature = compute_signature(secret_key, uri_path, nonce_check)
headers = {"API-Key": api_key, "API-Sign": signature}
params = {"nonce": nonce, "checksum": checksum, "payload": payload, "instrument": "doge_idr_spot"}

print(f'nonce: {nonce}')
print(f'checksum: {checksum}')
print(f'signature: {signature}')
print(f'nonce_check: {nonce_check}')
print(f'signature: {signature}')
print(f'headers: {headers}')
print(f'params: {params}')

response = requests.delete(
    URL_API_BITWYRE + uri_path,
    headers=headers,
    params=params,
)

print(response)
print(response.text)

nonce: 1744344073658899086
checksum: 135c3a22c18a0663c6082adbac1649042e5989663e4f2d779beb35038ebb891a
signature: 9c2297b968464e725f20d8431cef71270d9115b512e2933868e1ebd4c41eec41d2be6f434cd9f6c6660531dbc0fa9f842fdbae3b8c62f01dd90c41076781b7fa
nonce_check: abe8257a01bfc71708607f1b54375d943013c962e61192bbdbef33ecab084ee7
signature: 9c2297b968464e725f20d8431cef71270d9115b512e2933868e1ebd4c41eec41d2be6f434cd9f6c6660531dbc0fa9f842fdbae3b8c62f01dd90c41076781b7fa
headers: {'API-Key': '7rz0k4_00GoEtQW4td6mAM-7jxFCOLoFgKJzVD3x', 'API-Sign': '9c2297b968464e725f20d8431cef71270d9115b512e2933868e1ebd4c41eec41d2be6f434cd9f6c6660531dbc0fa9f842fdbae3b8c62f01dd90c41076781b7fa'}
params: {'nonce': 1744344073658899086, 'checksum': '135c3a22c18a0663c6082adbac1649042e5989663e4f2d779beb35038ebb891a', 'payload': '{"order_ids": ["5b88198c-3894-4748-8e35-31d9099499f6"], "qtys": [1], "instrument": "doge_idr_spot"}', 'instrument': 'doge_idr_spot'}
<Response [200]>
{
  "error": [], 
  "result": []
}

